In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/mnt', force_remount=True)
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

Mounted at /content/mnt


FileExistsError: ignored

In [2]:
#!pip install --target=$nb_path darts==0.19
#!pip install --target=$nb_path pyyaml==5.4.1

  Using cached darts-0.19.0-py3-none-any.whl (370 kB)
  Using cached torch-1.11.0-cp37-cp37m-manylinux1_x86_64.whl (750.6 MB)
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached lightgbm-3.3.2-py3-none-manylinux1_x86_64.whl (2.0 MB)
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
  Using cached xarray-0.20.2-py3-none-any.whl (845 kB)
  Using cached statsforecast-0.5.5-py3-none-any.whl (35 kB)
  Using cached nfoursid-1.0.0-py3-none-any.whl (16 kB)
  Using cached statsmodels-0.13.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.8 MB)
  Using cached prophet-1.0.1-py3-none-any.whl
  Using cached pmdarima-1.8.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.4 MB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
  Using cached matplotlib-3.5.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached pystan-2.19.1.1-cp37-cp37m-manylinux1_x86_64.wh

  Using cached PyYAML-5.4.1-cp37-cp37m-manylinux1_x86_64.whl (636 kB)


In [2]:
!pip install darts==0.19


  Using cached darts-0.19.0-py3-none-any.whl (370 kB)
  Using cached matplotlib-3.5.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached nfoursid-1.0.0-py3-none-any.whl (16 kB)
  Using cached statsforecast-0.5.5-py3-none-any.whl (35 kB)
  Using cached prophet-1.0.1-py3-none-any.whl
  Using cached holidays-0.13-py3-none-any.whl (172 kB)
  Using cached pmdarima-1.8.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.4 MB)
  Using cached ipython-7.33.0-py3-none-any.whl (793 kB)
  Using cached tbats-1.1.0-py3-none-any.whl (43 kB)
  Using cached statsmodels-0.13.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.8 MB)
  Using cached pytorch_lightning-1.6.3-py3-none-any.whl (584 kB)
  Using cached prompt_toolkit-3.0.29-py3-none-any.whl (381 kB)
  Using cached fonttools-4.33.3-py3-none-any.whl (930 kB)
  Using cached cmdstanpy-0.9.68-py3-none-any.whl (49 kB)
  Using cached ujson-5.2.0-cp37-cp37m-manylinux_2_17_x86_64.man

In [5]:
!pip install pyyaml==5.4.1

  Using cached PyYAML-5.4.1-cp37-cp37m-manylinux1_x86_64.whl (636 kB)
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import darts

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [4]:
import pickle
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error as mse
from torch.utils.data import DataLoader
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from darts import TimeSeries

torch.manual_seed(3407)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

PREFIX = "/content/mnt/MyDrive/he"

In [5]:
from darts.models import TFTModel

In [6]:
from torch.utils.data import Dataset

# pure copy
class HEDataset(Dataset):
    def __init__(self, data, pca=None, mean=None, std=None):
        print("Reduce dim")
        if not pca:
            pca = PCA(n_components=N_COMPONENTS)
            pca.fit(data)
            print(pca.explained_variance_)
        self.pca = pca
        transformed_data = pca.transform(data)
        timeframes = torch.Tensor(transformed_data)
        
        print("Data norm")
        if mean is None:
            mean = timeframes.mean(0)
        self.mean = mean
        print(self.mean)
        if std is None:
            std = timeframes.std(0)
        self.std = std
        transformed_data = timeframes - self.mean
        transformed_data = transformed_data / self.std
        self.original = transformed_data # todo debug only
        print("Unfold data into blocks")
        self.srcs = transformed_data[:-1].unfold(0, INPUT_STEP, 1).to(device)   # leave the last one for the last tgts
        self.tgts = torch.unsqueeze(transformed_data[INPUT_STEP:], 2).to(device)
        assert self.srcs.shape[0] == self.tgts.shape[0]
        step = self.srcs.shape[0] // 2
        assert torch.equal(self.srcs[step, :, -1].detach().unsqueeze(1), self.tgts[step-1, :, :].detach())
        print(self.srcs.shape, self.tgts.shape)

    def __getitem__(self, idx):
        src = self.srcs[idx, :, :]
        tgt = self.tgts[idx, :, :]
        return src, tgt
    
    def __len__(self):
        return self.srcs.shape[0]

In [12]:
with open(f'{PREFIX}/train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)

with open(f'{PREFIX}/val_data.pkl', 'rb') as f:
    val_data = pickle.load(f)   # todo change to val_data = pickle.load(f)

with open(f'{PREFIX}/test_data.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [13]:
# pure copy
N_COMPONENTS = 3
INPUT_STEP = 10
OUTPUT_STEP = 1
BATCH_SIZE = 8

In [14]:
train_dataset = HEDataset(train_data)

Reduce dim
[1.09692232e+06 2.74428237e+00 1.57192404e+00]
Data norm
tensor([ 1.9037e-07, -1.2394e-10, -3.0985e-10])
Unfold data into blocks
torch.Size([984911, 3, 10]) torch.Size([984911, 3, 1])


In [15]:
val_dataset = HEDataset(val_data, train_dataset.pca, train_dataset.mean, train_dataset.std)

Reduce dim
[1.99267286e+05 4.28486188e-06 2.58659812e-06]
Data norm
tensor([-4.7593e-08, -1.2103e-13, -3.4797e-13])
Unfold data into blocks
torch.Size([984911, 3, 10]) torch.Size([984911, 3, 1])


In [16]:
t=[0]*train_dataset.original.shape[1]
for component in range(train_dataset.original.shape[1]):
    t[component] = TimeSeries.from_values(train_dataset.original[:,component].numpy())

In [17]:
from itertools import compress

In [18]:
input_chunk_length = 10
forecast_horizon = 1


In [19]:
t[1]

<TimeSeries (DataArray) (time: 984921, component: 1, sample: 1)>
array([[[-0.23679712]],

       [[-0.35205865]],

       [[-4.579351  ]],

       ...,

       [[-0.07063782]],

       [[-0.1934466 ]],

       [[ 0.02362802]]], dtype=float32)
Coordinates:
  * time       (time) int64 0 1 2 3 4 5 ... 984916 984917 984918 984919 984920
  * component  (component) <U1 '0'
Dimensions without coordinates: sample

In [ ]:
models = []
for i in range(len(t)):
  print(i)
  my_model = TFTModel(
    input_chunk_length=input_chunk_length,
    output_chunk_length=forecast_horizon,
    hidden_size=64,
    lstm_layers=1,
    num_attention_heads=4,
    dropout=0.1,
    batch_size=16,
    n_epochs=3,
    add_relative_index=False,
    add_encoders=None,
    likelihood=None,
    loss_fn=nn.MSELoss(),
    random_state=42,
    pl_trainer_kwargs={'enable_progress_bar': True}
  )
  mask = np.arange(0, len(t)) != i
  future_covariates = list(compress(t, mask.tolist()))
  stacked_future_covariate = future_covariates[0]
  for j in range (1, len(future_covariates)):
    stacked_future_covariate = stacked_future_covariate.stack(future_covariates[j])
  my_model.fit(t[i], future_covariates=stacked_future_covariate)
  models.append(my_model)

0


[2022-05-14 09:02:39,488] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 984911 samples.
[2022-05-14 09:02:39,488] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 984911 samples.
2022-05-14 09:02:39 darts.models.forecasting.torch_forecasting_model INFO: Train dataset contains 984911 samples.
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'loss_fn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_fn'])`.
  f"Attribute {k!r} is an instance of `nn.Module` and is already saved during checkpointing."
[2022-05-14 09:02:39,608] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 32-bits; casting model to float32.
[2022-05-14 09:02:39,608] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 32-bits; casting mod

Training: 0it [00:00, ?it/s]

In [ ]:
for i, model in enumerate(models):
  model.save_model(f'{PREFIX}/model_{i}')

In [ ]:
# Probabilistic prediction
model.predict(*args, **kwargs, num_samples=200)

# Deterministic prediction

In [ ]:
# todo increase forcast_horizon